In [5]:
%reload_ext autoreload
%autoreload 2

import os
import re
import h5py

import numpy as np

# Check

In [23]:
data_dir = '/n/holystore01/LABS/iaifi_lab/Users/yitians/dm21cm/data/zf002/data'
os.listdir(data_dir)

['phot_dep.h5',
 'phot_prop.h5',
 'pbhacc_rates',
 'elec_dep.h5',
 'pwave_hmf_summed_rate.h5',
 'archive',
 'pbhhr-a0.999.h5',
 'abscissas.h5',
 'pwave_hmf_summed_rate_mc1e11.h5',
 'elec_scat.h5',
 'pbhhr-a0.000.h5',
 'phot_scat.h5']

In [24]:
! ls $data_dir/pbhhr*

/n/holystore01/LABS/iaifi_lab/Users/yitians/dm21cm/data/zf002/data/pbhhr-a0.000.h5
/n/holystore01/LABS/iaifi_lab/Users/yitians/dm21cm/data/zf002/data/pbhhr-a0.999.h5


In [22]:
! mv $data_dir/pbhhr-a0.000-merged.h5 $data_dir/pbhhr-a0.000.h5

In [18]:
with h5py.File(os.path.join(data_dir, 'pbhhr-a0.000-extras.h5'), 'r') as f:
    print(f.keys())

<KeysViewHDF5 ['log10m13.8750', 'log10m14.0625', 'log10m14.1250', 'log10m14.1875', 'log10m14.3125', 'log10m14.3750', 'log10m14.4375', 'log10m14.6250']>


# Increase float precision

In [12]:
h5_filename = os.path.join(data_dir, 'pbhhr-a0.999-merged.h5')
tmp_filename = h5_filename + ".tmp"

with h5py.File(h5_filename, "r") as f_in, h5py.File(tmp_filename, "w") as f_out:
    for group_name in f_in.keys():
        # Extract the float from the group name
        match = re.match(r"log10m([-+]?[0-9]*\.?[0-9]+)", group_name)
        if not match:
            print(f"Skipping non-matching group: {group_name}")
            continue

        val = float(match.group(1))
        new_name = f"log10m{val:.4f}"

        # Copy the group into the new file under the new name
        print(f"Renaming {group_name} to {new_name}")
        f_in.copy(group_name, f_out, name=new_name)

    # Also copy file-level attributes if needed
    for k, v in f_in.attrs.items():
        f_out.attrs[k] = v

    # Replace the original file
    os.replace(tmp_filename, h5_filename)
    print("Renaming complete.")


Renaming log10m13.2500 to log10m13.2500
Renaming log10m13.5000 to log10m13.5000
Renaming log10m13.7500 to log10m13.7500
Renaming log10m14.0000 to log10m14.0000
Renaming log10m14.2500 to log10m14.2500
Renaming log10m14.375 to log10m14.3750
Renaming log10m14.450 to log10m14.4500
Renaming log10m14.499 to log10m14.4990
Renaming log10m14.5000 to log10m14.5000
Renaming log10m14.501 to log10m14.5010
Renaming log10m14.550 to log10m14.5500
Renaming log10m14.625 to log10m14.6250
Renaming log10m14.7500 to log10m14.7500
Renaming log10m15.0000 to log10m15.0000
Renaming log10m15.2500 to log10m15.2500
Renaming log10m15.5000 to log10m15.5000
Renaming log10m15.7500 to log10m15.7500
Renaming log10m16.0000 to log10m16.0000
Renaming log10m16.2500 to log10m16.2500
Renaming log10m16.5000 to log10m16.5000
Renaming log10m16.7500 to log10m16.7500
Renaming log10m17.0000 to log10m17.0000
Renaming log10m17.2500 to log10m17.2500
Renaming log10m17.5000 to log10m17.5000
Renaming log10m17.7500 to log10m17.7500
Renami

# Merge

In [10]:
def merge_h5_files(src_a, src_b, dst_path, overwrite=False, copy_file_attrs=True):
    """
    Merge top-level items (groups/datasets) from src_a and src_b into dst_path.
    Raises MergeConflict if there are duplicate top-level keys or duplicate file attrs.
    """
    # 'w-' fails if file exists; 'w' overwrites.
    mode = "w" if overwrite else "w-"

    with h5py.File(src_a, "r") as fa, h5py.File(src_b, "r") as fb, h5py.File(dst_path, mode) as fout:
        keys_a = set(fa.keys())
        keys_b = set(fb.keys())

        dup_keys = sorted(keys_a & keys_b)
        if dup_keys:
            raise ValueError(f"Duplicate top-level keys: {dup_keys}")

        # Copy A then B (order preserved as in file iteration)
        for k in fa.keys():
            print(f"Copying {k} from {src_a}")
            fa.copy(fa[k], fout, name=k)
        for k in fb.keys():
            print(f"Copying {k} from {src_b}")
            fb.copy(fb[k], fout, name=k)

        if copy_file_attrs:
            # Copy file-level attributes from A then B; raise if any attr collides
            for k, v in fa.attrs.items():
                fout.attrs[k] = v
            for k, v in fb.attrs.items():
                if k in fout.attrs:
                    raise ValueError(f"Duplicate file-level attribute: '{k}'")
                fout.attrs[k] = v

In [19]:
merge_h5_files(
    os.path.join(data_dir, 'pbhhr-a0.000.h5'),
    os.path.join(data_dir, 'pbhhr-a0.000-extras.h5'),
    os.path.join(data_dir, 'pbhhr-a0.000-merged.h5'),
)

Copying log10m13.2500 from /n/holystore01/LABS/iaifi_lab/Users/yitians/dm21cm/data/zf002/data/pbhhr-a0.000.h5
Copying log10m13.5000 from /n/holystore01/LABS/iaifi_lab/Users/yitians/dm21cm/data/zf002/data/pbhhr-a0.000.h5
Copying log10m13.7500 from /n/holystore01/LABS/iaifi_lab/Users/yitians/dm21cm/data/zf002/data/pbhhr-a0.000.h5
Copying log10m14.0000 from /n/holystore01/LABS/iaifi_lab/Users/yitians/dm21cm/data/zf002/data/pbhhr-a0.000.h5
Copying log10m14.2500 from /n/holystore01/LABS/iaifi_lab/Users/yitians/dm21cm/data/zf002/data/pbhhr-a0.000.h5
Copying log10m14.5000 from /n/holystore01/LABS/iaifi_lab/Users/yitians/dm21cm/data/zf002/data/pbhhr-a0.000.h5
Copying log10m14.7500 from /n/holystore01/LABS/iaifi_lab/Users/yitians/dm21cm/data/zf002/data/pbhhr-a0.000.h5
Copying log10m15.0000 from /n/holystore01/LABS/iaifi_lab/Users/yitians/dm21cm/data/zf002/data/pbhhr-a0.000.h5
Copying log10m15.2500 from /n/holystore01/LABS/iaifi_lab/Users/yitians/dm21cm/data/zf002/data/pbhhr-a0.000.h5
Copying lo